<a href="https://colab.research.google.com/github/jpmachinelearning/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J/blob/main/src/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyGithub
!python -m spacy download es_core_news_lg
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 3.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 10.1 MB/s eta 0:00:00


In [13]:
import emoji
import re
import spacy
import nltk
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from google.colab import files, userdata
from github import Github, Auth, GithubException
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# Descargas NLTK (ejecutar una vez)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab')
nltk.download('stopwords', quiet=True)

STOP_ES = set(stopwords.words('spanish'))
STOP_EN = set(stopwords.words('english'))

nlp = spacy.blank("es")
nlp2 = spacy.load("es_core_news_lg")

if "sentencizer" not in nlp.pipe_names:
    nlp.add_pipe("sentencizer")

# Inicializar archivo de log (sobrescribe en cada ejecución)
with open('log_file.txt', 'w', encoding='utf-8') as _log_init:
    _log_init.write(f"Log iniciado: {datetime.datetime.utcnow().isoformat()} UTC\n")
    _log_init.write("="*120 + "\n\n")

INLINE_CODE = re.compile(r'`[^`]+`')
FENCED_CODE = re.compile(r'```(.+?)```', re.DOTALL)
CODE_KEYWORDS = re.compile(r'\b(def|class|import|from|return|console\.log|printf|#include|std::|->|=>)\b')
BRACES_SEMICOLON = re.compile(r'[{};=<>]')
IMG_TAG = re.compile(r'<img\b[^>]*>', re.IGNORECASE)
HTML_TAG = re.compile(r'<\/?\w+[^>]*>', re.IGNORECASE)
URL_LARGE = re.compile(r'https?://\S{30,}', re.IGNORECASE)
ASSET_DOMAINS = re.compile(r'(github\.com|githubusercontent\.com|assets/|cdn\.)', re.IGNORECASE)

MAPEO_LEMAS = {
  "hi": "hola",
  "Ademas": "además",
  "ademas": "además",
  "qutarir": "quitar",
  "qutacer": "quitar",
  "qutaria": "quitar",
  "qutaría": "quitar",
  "añadiais": "añadir",
  "válir": "válido",
  "gracia": "gracias",
  "modul": "módulo",
  "finar": "final",
  "somar": "sumar",
  "deveria": "deber",
  "devería": "deber",
  "estaria": "estar",
  "estáría": "estar",
  "configurancion": "configuración",
  "temrinos": "términos",
  "despés": "después",
  "propois": "propios",
  "válir": "valer",
  "viudedad": "viudedad",
  "reav": "revisar",
  "qeu": "que",
  "pusistar": "pusiste",
  "modificaco": "modificado",
  "descárgatar": "descargar",
  "fuentir": "fuente",
  "invoizar": "facturar",
  "informacion": "información",
  "subsana": "subsanar",
  "eskema": "esquema",
  }

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/tmp/ipython-input-3420185403.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  _log_init.write(f"Log iniciado: {datetime.datetime.utcnow().isoformat()} UTC\n")


In [7]:

# ==========================================
# 1. OBTENCIÓN DE DATOS
# ==========================================
try:
    TOKEN = userdata.get('GITHUB_TOKEN')
    # Configuración para Git por comandos
    USER_GITHUB = "jpmachinelearning" # Tu usuario de GitHub
    EMAIL_GITHUB = "tu_email@ejemplo.com" # Cambia esto por tu email de GitHub
except userdata.SecretNotFoundError:
    print("❌ Error: Asegúrate de crear el secreto 'GITHUB_TOKEN' en el panel de la izquierda.")

# Repositorio fuente para extracción
REPO_NAME = 'OCA/l10n-spain'

# Repositorio destino para guardar la data (formato para Git)
DEST_REPO_NAME = 'U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J'
REPO_URL = f"https://{TOKEN}@github.com/{USER_GITHUB}/{DEST_REPO_NAME}.git"

# Configuración de archivos
nombre_archivo = 'datos_github.csv'
ruta_local = os.path.join('/content/sample_data', nombre_archivo)

# Límite de registros
LIMITE_REGISTROS = 200

# ==========================================
# 2. CONFIGURACIÓN DE GIT (PULL)
# ==========================================
# Clonamos el repo si no existe, o hacemos pull si ya está
%cd /content/
if not os.path.exists(DEST_REPO_NAME):
    print(f"🚀 Clonando repositorio {DEST_REPO_NAME}...")
    !git clone {REPO_URL}
else:
    print(f"🔄 Sincronizando repositorio (git pull)...")
    %cd {DEST_REPO_NAME}
    !git pull origin main
    %cd /content/

# Configurar identidad de Git
!git config --global user.email "{EMAIL_GITHUB}"
!git config --global user.name "{USER_GITHUB}"

# ==========================================
# 2. CONEXIÓN API (Extracción)
# ==========================================
try:
    auth = Auth.Token(TOKEN)
    g = Github(auth=auth)

    repo = g.get_repo(REPO_NAME)
    print(f"✅ Conectado exitosamente al proyecto: {repo.full_name}")

    data = []

    # ==========================================
    # 3. EXTRACCIÓN DE ISSUES (RF-01)
    # ==========================================
    print(f"\n--- Extrayendo hasta {LIMITE_REGISTROS} Issues ---")
    issues = repo.get_issues(state='all', sort='created', direction='desc')
    count = 0

    for item in issues:
        if count >= LIMITE_REGISTROS: break
        if item.pull_request is not None: continue

        print(f"📥 Procesando Issue #{item.number}...")
        comments = [c.body for c in item.get_comments() if c.body]

        data.append({
            'tipo': 'Issue',
            'numero': item.number,
            'comentarios': " [SEP] ".join(comments)
        })
        count += 1

    # ==========================================
    # 4. EXTRACCIÓN DE PULL REQUESTS (RF-01)
    # ==========================================
    print(f"\n--- Extrayendo hasta {LIMITE_REGISTROS} Pull Requests ---")
    pulls = repo.get_pulls(state='all', sort='created', direction='desc')
    count = 0

    for pr in pulls:
        if count >= LIMITE_REGISTROS: break

        print(f"📥 Procesando PR #{pr.number}...")
        comments = [c.body for c in pr.get_issue_comments() if c.body]

        data.append({
            'tipo': 'Pull Request',
            'numero': pr.number,
            'titulo': item.title,
            'descripcion': item.body,
            'comentarios': " [SEP] ".join(comments)
        })
        count += 1

    # ==========================================
    # 5. GUARDAR RESULTADOS LOCALMENTE
    # ==========================================
    if not data:
        print("⚠️ No se encontraron datos.")
    else:
        df = pd.DataFrame(data)

        # Guardar en /content/sample_data/datos_github.csv
        df.to_csv(ruta_local, index=False, encoding='utf-8-sig')

        print("-" * 30)
        print(f"✨ RESULTADOS DE LA RECOLECCIÓN:")
        print(f"📁 Guardado local en: {ruta_local}")
        print(f"📊 Total de filas (Issues + PRs): {len(df)}")
        print("-" * 30)

        # ==========================================
        # 6. ENVIAR A REPOSITORIO USANDO GIT PUSH
        # ==========================================
        print(f"\n📦 Moviendo archivo al repositorio y subiendo cambios...")

        # Asegurar que la carpeta data existe en el repo clonado
        repo_path = f"/content/{DEST_REPO_NAME}/data"
        os.makedirs(repo_path, exist_ok=True)

        # Copiar de sample_data al repositorio
        !cp {ruta_local} {repo_path}/{nombre_archivo}

        # Comandos de Git para subir
        %cd /content/{DEST_REPO_NAME}
        !git add data/{nombre_archivo}
        !git commit -m "Actualización de datos_github.csv via Git Pull/Push desde Colab"
        !git push origin main

        print(f"\n✅ Proceso completado. Archivo disponible en GitHub y localmente en sample_data.")
        %cd /content/

except Exception as e:
    print(f"❌ Error: {e}")

# ==========================================
# 6. ENVIAR A REPOSITORIO (CON MANEJO DE CONFLICTOS)
# ==========================================
print(f"\n🚀 Sincronizando y subiendo cambios a GitHub...")

# Ruta del repo y carpeta data
repo_path = f"/content/{DEST_REPO_NAME}"
data_folder = os.path.join(repo_path, "data")
os.makedirs(data_folder, exist_ok=True)

# Copiar el archivo fresco de sample_data al repo clonado
!cp {ruta_local} {data_folder}/{nombre_archivo}

%cd {repo_path}

# 1. Preparar el commit local
!git add data/{nombre_archivo}
# Usamos un commit con fecha para diferenciar
import datetime
timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
!git commit -m "Auto-update data: {timestamp}"

# 2. Intentar traer cambios remotos con Rebase
# La estrategia '-X ours' le dice a Git: "Si hay conflicto en una línea,
# mantén lo que acabo de generar en este Colab".
print("🔄 Revisando si hay cambios externos (Pull Rebase)...")
status = os.system("git pull --rebase -X ours origin main")

if status == 0:
    # 3. Empujar los cambios
    print("📤 Subiendo cambios finales...")
    push_status = os.system("git push origin main")
    if push_status == 0:
        print(f"✅ ¡Éxito! Datos sincronizados en el repositorio.")
    else:
        print("❌ Error al hacer push. Revisa permisos del Token.")
else:
    print("⚠️ Hubo un problema al sincronizar. Intentando limpiar estado de Git...")
    !git rebase --abort

%cd /content/

/content
🚀 Clonando repositorio U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J...
Cloning into 'U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 46 (delta 15), reused 4 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 217.96 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (15/15), done.
✅ Conectado exitosamente al proyecto: OCA/l10n-spain

--- Extrayendo hasta 200 Issues ---
📥 Procesando Issue #4793...
📥 Procesando Issue #4760...
📥 Procesando Issue #4743...
📥 Procesando Issue #4681...
📥 Procesando Issue #4677...
📥 Procesando Issue #4666...
📥 Procesando Issue #4652...
📥 Procesando Issue #4650...
📥 Procesando Issue #4649...
📥 Procesando Issue #4642...
📥 Procesando Issue #4641...
📥 Procesando Issue #4639...
📥 Procesando Issue #4637...
📥 Procesando Issue #4632...
📥 Procesando Issue #4631...
📥 Procesando Issue #4626...
📥 Procesando Issu

In [8]:
file_path = '/content/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J/data/datos_github.csv'
#data = pd.read_csv(file_path, sep='\s+', header=None)
data = pd.read_csv(file_path)
data

,tipo,numero,comentarios,titulo,descripcion
0,Issue,4793,Siendo tratado en #4784,NaN,NaN
1,Issue,4760,"Además, hemos podido comprobar que el mismo pr...",NaN,NaN
2,Issue,4743,NaN,NaN,NaN
3,Issue,4681,Hola.\n\nAgradecería si alguien puede confirma...,NaN,NaN
4,Issue,4677,NaN,NaN,NaN
...,...,...,...,...,...
395,Pull Request,4579,`\r\n if(!result.verifactu_qr){\r\n ...,[15.0][IMP]l10n_es_facturae: add facturae_hide...,This PR adds to l10n_es_facturae the option to...
396,Pull Request,4577,/ocabot merge major [SEP] On my way to merge t...,[15.0][IMP]l10n_es_facturae: add facturae_hide...,This PR adds to l10n_es_facturae the option to...
397,Pull Request,4576,/ocabot merge patch [SEP] On my way to merge t...,[15.0][IMP]l10n_es_facturae: add facturae_hide...,This PR adds to l10n_es_facturae the option to...
398,Pull Request,4575,/ocabot merge patch [SEP] This PR looks fantas...,[15.0][IMP]l10n_es_facturae: add facturae_hide...,This PR adds to l10n_es_facturae the option to...


In [9]:
data.count()

,0
tipo,400
numero,400
comentarios,357
titulo,200
descripcion,200


In [10]:
data.shape

(400, 5)

In [11]:

# ==========================================
# 2. PREPROCESADO
# ==========================================

def ensure_text_input(corpus):
    if isinstance(corpus, str):
        return corpus
    # pandas Series o DataFrame cell
    if pd is not None and isinstance(corpus, pd.Series):
        # eliminar NaN y convertir a str
        parts = [str(x) for x in corpus.dropna().astype(str).tolist()]
        return "\n\n".join(parts)
    # lista/tupla de strings
    if isinstance(corpus, (list, tuple)):
        parts = [str(x) for x in corpus if x is not None]
        return "\n\n".join(parts)
    # fallback: intentar str()
    return str(corpus)


def apply_mapeo_and_simple_grammar(text, corrections=None):
    """
    Aplica MAPEO_LEMAS sobre tokens y corrige patrones gramaticales puntuales.
    Si se pasa 'corrections' (lista), añade tuplas (original, reemplazo).
    """
    if not isinstance(text, str) or not text:
        return text
    # 1) Normalizar tokens simples usando MAPEO_LEMAS (mantener espacios)
    def replace_token_match(m):
        tok_orig = m.group(0)
        tok = tok_orig.lower()
        tok_repl = MAPEO_LEMAS.get(tok, tok)
        # registrar si hubo cambio real y se pasó lista
        if corrections is not None and tok_repl != tok:
            corrections.append((tok_orig, tok_repl))
        return tok_repl
    pattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in MAPEO_LEMAS.keys()) + r')\b', flags=re.IGNORECASE)
    text = pattern.sub(lambda m: replace_token_match(m), text)
    # 2) Reglas gramaticales puntuales (ejemplos seguros y no intrusivos)
    #    - "ningún excepción" -> "ninguna excepción"
    def replace_ningun_excepcion(m):
        orig = m.group(0)
        repl = 'ninguna excepción'
        if corrections is not None:
            corrections.append((orig, repl))
        return repl
    text = re.sub(r'\bningún\s+excepción\b', replace_ningun_excepcion, text, flags=re.IGNORECASE)
    #    - corregir dobles espacios
    text = re.sub(r'\s{2,}', ' ', text)
    text = text.strip()
    return text


# --- Sustituciones finales robustas: URLs -> 'url' y números -> 'num' ---
def apply_placeholders_preserve_punct(text: str, replacements=None) -> str:
    if not isinstance(text, str) or not text:
        return text
    # URL pattern (prioriza detección de URL completa)
    url_pattern = re.compile(r'(?P<prefix>[\(\[\{<"\']?)'r'(?P<url>(?:https?[:/]{0,3}|//|www\.)'r'[A-Za-z0-9\-]+'r'(?:\.[A-Za-z0-9\-]+)+'r'(?:[\/?#][^\s\)\]\}\>,.;:!?"\']*)?)'r'(?P<suffix>[\)\]\}>,.;:!?"\']*)',flags=re.IGNORECASE)
    def _url_sub(m):
        orig = m.group('url')
        if replacements is not None:
            replacements.append((orig, 'url'))
        return f"{m.group('prefix')}url{m.group('suffix')}"
    text = url_pattern.sub(_url_sub, text)
    # Número puro: preservar prefijo/sufijo de puntuación
    number_pattern = re.compile(r'(?<![A-Za-z0-9])'r'(?P<prefix>[\(\[\{<"\']?)'r'(?P<num>\d+)'r'(?P<suffix>[\)\]\}>,.;:!?"\']?)'r'(?![A-Za-z0-9])')
    def _num_sub(m):
        orig = m.group('num')
        if replacements is not None:
            replacements.append((orig, 'num'))
        return f"{m.group('prefix')}num{m.group('suffix')}"


    text = number_pattern.sub(_num_sub, text)
    return text


def preprocesado(doc):
  # Pipeline del prepocesado: normalización -> traducción de emoji -> clasificador (doble revisión 'und') / eliminación segmentos 'en', 'und' y 'code' ->
  # elimanción de typos -> cambio de numeros por placeholder 'num' y sitio web por 'url' ->
  # eliminación signos de puntuación -> tokenización -> eliminación de stopwords -> lematización


  ### Normalización
  #Normalizar entrada a texto (maneja None, NaN, Series, listas, etc.)
  doc = ensure_text_input(doc) # Si queda vacío, devolver resultado vacío y log mínimo
  if not doc or str(doc).strip() == "":
    return "", [["input_empty"]]
  doc = doc.lower()


  ### Traducción de emoji
  #doc = emoji.demojize(doc, language='es').replace(':', '').replace('_', ' ')

  # 1. Comprobamos si existen emojis en el documento mediante un condicional
  if emoji.emoji_count(doc) > 0:

      # 2. Definimos una función interna (callback) que solo procesa el emoji hallado
      def procesar_emoji_individual(chars, data_dict):
          # Traducimos el emoji a su nombre en español (ej: :cara_sonriente:)
          nombre_emoji = emoji.demojize(chars, language='es')

          # 3. Limpiamos los signos SOLO del nombre generado por demojize.
          # Esto permite que "cara_sonriente" pase a ser "cara sonriente"
          # para que el análisis de sentimiento/TF-IDF lo entienda mejor,
          # pero sin afectar a las URLs del resto del documento.
          nombre_limpio = nombre_emoji.replace(':', '').replace('_', ' ')

          # Devolvemos el texto con espacios para que no se pegue a otras palabras
          return f" {nombre_limpio} "

      # 4. Aplicamos el reemplazo selectivo
      doc = emoji.replace_emoji(doc, replace=procesar_emoji_individual)

  ### Elimanción de typos
  ortho_corrections = []
  doc = apply_mapeo_and_simple_grammar(doc, corrections=ortho_corrections)


  ### Clasificador (doble revision 'und') / eliminacion segmentos 'en', 'und' y 'code'
  def looks_like_code_line(line):
      if INLINE_CODE.search(line):
          return True
      if FENCED_CODE.search(line):
          return True
      if IMG_TAG.search(line) or HTML_TAG.search(line):
          return True
      if URL_LARGE.search(line) and ASSET_DOMAINS.search(line):
          return True
      if CODE_KEYWORDS.search(line):
          return True
      non_alpha = sum(1 for ch in line if not ch.isalpha() and not ch.isspace())
      if len(line) > 0 and (non_alpha / len(line)) > 0.25:
          return True
      if BRACES_SEMICOLON.search(line):
          return True
      return False

  def split_by_blank_and_guess(segment):
      parts = []
      groups = re.split(r'\n\s*\n', segment)
      for g in groups:
          lines = [ln for ln in g.splitlines() if ln.strip() != ""]
          if not lines:
              continue
          #FORZAR code si hay etiquetas img/html o URLs de assets en las líneas
          # (requiere que IMG_TAG, HTML_TAG, URL_LARGE, ASSET_DOMAINS estén definidos globalmente)
          if any(IMG_TAG.search(ln) or (URL_LARGE.search(ln) and ASSET_DOMAINS.search(ln)) or HTML_TAG.search(ln) for ln in lines):
              parts.append(('code', "\n".join(lines)))
              continue
          code_like_count = sum(1 for ln in lines if looks_like_code_line(ln) or ln.startswith('    ') or ln.startswith('\t'))
          prop = code_like_count / len(lines)
          if prop >= 0.4:
              code_lines = []
              text_lines = []
              for ln in lines:
                  tokens = [t.lower() for t in word_tokenize(ln) if t.isalpha()]
                  if len(tokens) >= 2:
                      es_count = sum(1 for t in tokens if t in STOP_ES)
                      if es_count / len(tokens) >= 0.25 or re.search(r'[áéíóúñÁÉÍÓÚÑ]', ln):
                          text_lines.append(ln)
                          continue
                  code_lines.append(ln)
              if code_lines:
                  parts.append(('code', "\n".join(code_lines)))
              if text_lines:
                  parts.append(('text', "\n".join(text_lines)))
          else:
              parts.append(('text', "\n".join(lines)))
      return parts

  def split_into_blocks_by_indentation_and_fences(corpus):
      """
      Acepta corpus de cualquier tipo; primero normaliza a string.
      Extrae fenced code y agrupa el resto por bloques separados por línea en blanco.
      """
      text = ensure_text_input(corpus)
      # --- NORMALIZACIÓN ADICIONAL: convertir separadores en "punto y aparte" ---
      # 1) [sep] o [ sep] -> punto y salto de línea doble
      text = re.sub(r'\[\s*sep\s*\]', '.\n\n', text, flags=re.IGNORECASE)
      # 2) punto seguido (". ") -> punto y salto de línea doble
      text = re.sub(r'\.\s+', '.\n\n', text)
      # ------------------------------------------------------------------------
      blocks = []
      cursor = 0
      for m in FENCED_CODE.finditer(text):
          pre = text[cursor:m.start()]
          if pre.strip():
              blocks.extend(split_by_blank_and_guess(pre))
          code_block = m.group(0)
          blocks.append(('code', code_block))
          cursor = m.end()
      tail = text[cursor:]
      if tail.strip():
          blocks.extend(split_by_blank_and_guess(tail))
      return blocks

  def detect_language_by_stopwords(text, min_tokens=1):
      """
      Detecta 'es', 'en', 'bilingual' o 'und'.
      Ajustes:
        - tokenización robusta (quita prefijos/sufijos no alfabéticos)
        - lista blanca ampliada (incluye 'finde')
        - heurística para frases cortas sin stopwords
        - filtra tokens de longitud 1 al contar stopwords
        - prioriza 'es' cuando la señal española es claramente mayor
        - condición 'bilingual' más estricta
      """
      txt_stripped = text.strip()
      # caso puntual muy corto (ej. "Gracias" o "Gracias.")
      if txt_stripped.lower() in {'gracias', 'gracias.'}:
          return 'es'
      # tokenización robusta: limpiar prefijos/sufijos no alfabéticos
      raw_tokens = word_tokenize(text)
      tokens = []
      for t in raw_tokens:
          t_clean = re.sub(r'^[^A-Za-zÁÉÍÓÚÑáéíóúñ]+|[^A-Za-zÁÉÍÓÚÑáéíóúñ]+$', '', t)
          if t_clean:
              tokens.append(t_clean.lower())
      # lista blanca para singletons y sufijos típicos
      SPANISH_SINGLETONS = {'gracias','hola','adjunto','captura','saludos','ok','listo','si','no','vale','finde'}
      SPANISH_SUFFIXES = ('ción','dad','mente','ado','ada','ico','ica','oso','osa')
      # 1) single token common spanish
      if len(tokens) == 1:
          single = tokens[0]
          if single in SPANISH_SINGLETONS or re.search(r'[áéíóúñ]', single):
              return 'es'
      # si hay pocos tokens, aplicar heurísticas adicionales (fallback temprano)
      if len(tokens) < min_tokens:
          if re.search(r'[áéíóúñÁÉÍÓÚÑ]', text):
              return 'es'
          low = text.lower()
          for kw in ('que','para','con','por','como','ejemplo','función','usar','gracias','alguna','novedad','adjunto','captura','finde'):
              if kw in low:
                  return 'es'
          return 'und'
      # filtrar tokens muy cortos para el conteo (evita ruido de 'a', 'I', etc.)
      tokens_for_count = [t for t in tokens if len(t) > 1]
      if not tokens_for_count:
          tokens_for_count = tokens
      # conteo de stopwords usando tokens filtrados
      es_count = sum(1 for t in tokens_for_count if t in STOP_ES)
      en_count = sum(1 for t in tokens_for_count if t in STOP_EN)
      es_ratio = es_count / len(tokens_for_count) if tokens_for_count else 0
      en_ratio = en_count / len(tokens_for_count) if tokens_for_count else 0
      # --- NUEVA REGLA PUNTUAL: preferir 'en' si hay URL/asset y señal mínima de inglés ---
      if (ASSET_DOMAINS.search(text) or re.search(r'https?://', text)) and en_ratio >= 0.05:
          return 'en'
      # ------------------------------------------------------------------------------
      # 2) heurística para frases cortas sin stopwords (casos como "Adjunto captura.")
      if len(tokens) >= 2 and es_count == 0 and en_count == 0:
          # aceptar si hay acento en alguno, sufijo típico español o palabra en whitelist
          if any(re.search(r'[áéíóúñ]', t) for t in tokens):
              return 'es'
          if any(t in SPANISH_SINGLETONS for t in tokens):
              return 'es'
          if any(t.endswith(SPANISH_SUFFIXES) for t in tokens):
              return 'es'
          # si no hay evidencia clara de inglés, asumir español de forma conservadora
          return 'es'
      # Priorizar español o inglés si la señal es claramente mayor
      if es_ratio > en_ratio and es_ratio >= 0.12:
          return 'es'
      if en_ratio > es_ratio and en_ratio >= 0.12:
          return 'en'
      # marcar bilingual solo si hay evidencia sólida en ambos idiomas
      if len(tokens_for_count) >= 3 and es_ratio >= 0.12 and en_ratio >= 0.12:
          return 'bilingual'
      # decisión por proporción de stopwords (criterio secundario)
      if es_ratio >= 0.12 and es_ratio > en_ratio:
          return 'es'
      if en_ratio >= 0.12 and en_ratio > es_ratio:
          return 'en'
      # fallback final por acentos
      if re.search(r'[áéíóúñÁÉÍÓÚÑ]', text):
          return 'es'
      return 'und'

  def classify_and_extract_strict(corpus, recursividad, window=3):
      """
      Versión ajustada:
        - Imprime la salida (category, spanish_with_code, code_blocks, items, details)
          antes de devolver el resultado.
        - Reconstruye (recomposición) un nuevo texto que solo incluye secciones
          consideradas españolas (lang == 'es' o 'bilingual').
        - Para elementos eliminados (lang in {'und','code','en'} o type == 'code'),
          imprime "Dato borrado:" seguido del item.
      """
      log = []
      text = ensure_text_input(corpus)
      blocks = split_into_blocks_by_indentation_and_fences(text)
      items = []
      for typ, content in blocks:
          if typ == 'code':
              m = FENCED_CODE.search(content)
              inner = m.group(1) if m else content
              items.append({'type': 'code', 'text': inner.strip(), 'lang': 'code'})
          else:
              doc = nlp(content)
              for sent in doc.sents:
                  s = sent.text.strip()
                  if not s:
                      continue
                  lang = detect_language_by_stopwords(s)
                  items.append({'type': 'text', 'text': s, 'lang': lang})
      total = len(items)
      code_count = sum(1 for it in items if it['type'] == 'code')
      es_count = sum(1 for it in items if it['lang'] == 'es')
      en_count = sum(1 for it in items if it['lang'] == 'en')
      if total == 0:
          category = "texto ingles 100%"
      else:
          prop_code = code_count / total
          prop_es = es_count / total
          prop_en = en_count / total
          if prop_code >= 0.95:
              category = "texto codigo de programacion 100%"
          elif prop_es >= 0.95 and code_count == 0:
              category = "texto español 100%"
          elif prop_en >= 0.95 and code_count == 0:
              category = "texto ingles 100%"
          elif prop_es > 0.5 and code_count > 0:
              category = "texto español + codigo de programacion"
          else:
              category = "texto bilingue"
      spanish_related = []
      for i, it in enumerate(items):
          if it['type'] == 'code':
              start = max(0, i - window)
              end = min(len(items) - 1, i + window)
              for j in range(start, end + 1):
                  cand = items[j]
                  if cand['type'] == 'text' and cand['lang'] == 'es':
                      spanish_related.append(cand['text'])
      if not spanish_related:
          tech_keywords = {'función','variable','clase','método','archivo','ejemplo','parámetro','argumento','instalar','importar','usar'}
          for it in items:
              if it['type'] == 'text' and it['lang'] == 'es':
                  tokens = set(t.lower() for t in word_tokenize(it['text']) if t.isalpha())
                  if tokens & tech_keywords or re.search(r'[áéíóúñ]', it['text']):
                      spanish_related.append(it['text'])
      seen = set()
      spanish_with_code = []
      for s in spanish_related:
          if s not in seen:
              seen.add(s)
              spanish_with_code.append(s)
      code_blocks = [it['text'] for it in items if it['type'] == 'code']
      for it in items:
          log.append(f"{it['type']} | {it['lang']} | {it['text']}")
      log.append(f"Detalles: total_items: {total} | code_items: {code_count} | spanish_items: {es_count} | english_items: {en_count}")
      # --- Recomposición del texto original manteniendo solo español ---
      # Incluir items cuya etiqueta de idioma sea 'es' o 'bilingual' y que no sean código.
      recomposed_parts = []
      for it in items:
        # condición para conservar: tipo text y lang es o bilingual
        if recursividad:
          if it['type'] == 'text' and it['lang'] in {'es', 'bilingual'}:
            recomposed_parts.append(it['text'])
          else:
            log.append(f"Dato borrado: {it['type']} | {it['lang']} | {it['text']}")
        elif it['type'] == 'text' and it['lang'] in {'es', 'bilingual', 'und'}:
          recomposed_parts.append(it['text'])
        else:
          # imprimir mensaje de borrado para los elementos no conservados
          log.append(f"Dato borrado: {it['type']} | {it['lang']} | {it['text']}")
      # unir con un espacio entre oraciones/secciones
      recomposed_text = " ".join(p.strip() for p in recomposed_parts if p.strip())
      log.append(f"recomposed_text: {recomposed_text}")
      # devolver la recomposición y el log
      return recomposed_text, log
  logs_for_flat =[]
  res1, log1 = classify_and_extract_strict(doc, recursividad=False)
  logs_for_flat.append(log1)
  res2, log2 = classify_and_extract_strict(res1, recursividad=True)
  res2 = (res2 or "").strip()
  # normalizar res2 (recomposed_text)
  if not res2:
    res2 = ""


  ### Cambio de numeros por placeholder 'num' y sitio web por 'url' (desactivado)
  placeholder_changes = []
  #res2 = apply_placeholders_preserve_punct(res2, replacements=placeholder_changes)


  if log2:
    logs_for_flat.append("__Recursividad__")
  logs_for_flat.append(log2)
  # aplanar (flatten) y asegurar que cada elemento sea string y sin saltos iniciales/finales
  logs = []
  for part in logs_for_flat:
      if isinstance(part, (list, tuple)):
          for line in part:
              logs.append(str(line).strip())
      else:
          logs.append(str(part).strip())
  # eliminar entradas vacías si las hubiera
  logs = [l for l in logs if l]
  # --- Escribir logs, cambios placeholders y ortográficos en archivo log_file.txt ---
  timestamp = datetime.datetime.utcnow().isoformat()
  try:
      with open('log_file.txt', 'a', encoding='utf-8') as lf:
          # si hay logs generales ya calculados (variable logs existente), escribirlos
          if logs:
              lf.write(f"# Entrada procesada {timestamp} UTC\n")
              for line in logs:
                  lf.write(line + "\n")
              lf.write("\n")
          # 1) Correcciones ortográficas (si las hay)
          if ortho_corrections:
              lf.write(f"__Corrector Ortográfico__{timestamp}\n")
              for orig, repl in ortho_corrections:
                  lf.write(f"'{orig}' se corrige por '{repl}'\n")
              lf.write("\n")
          # 2) Cambios por placeholders (si los hay)
          if placeholder_changes:
              lf.write(f"__Cambio por Placeholder__{timestamp}\n")
              for orig, repl in placeholder_changes:
                  lf.write(f"'{orig}' se cambio por '{repl}'\n")
              lf.write("\n")
  except Exception:
      # no interrumpir el pipeline por errores de logging
      pass

  def limpieza_selectiva_tecnica(doc):
      """
      Limpia signos de puntuación respetando:
      - URLs completas (https://www.aeat.es)
      - Versiones (16.0, 17.0.1)
      - Guiones entre texto/números (l10n-es, factura-e, v18-beta)
      """
      # 1. Normalización de espacios y saltos de línea
      doc = doc.replace('\\n', ' ').replace('\n', ' ')
      # 2. PROTEGER URLs
      urls = re.findall(r'https?://\S+', doc)
      for i, url in enumerate(urls):
          doc = doc.replace(url, f' TOKEN_URL_{i} ')
      # 3. PROTEGER VERSIONES (Puntos entre números)
      versiones = re.findall(r'\d+\.\d+(?:\.\d+)*', doc)
      for i, ver in enumerate(versiones):
          doc = doc.replace(ver, f' TOKEN_VER_{i} ')
      # 4. LIMPIEZA DE SIGNOS (Excepto el guion por ahora)
      # Borramos: ¡ ! ¿ ? [ ] ( ) { } | # * , ; : " ' » « @
      signos_a_borrar = r'[¡!¿?\[\](){}|#*,\;:\"\'»«@]'
      doc = re.sub(signos_a_borrar, ' ', doc)
      # 5. TRATAMIENTO DE PUNTOS Y GUIONES RESTANTES
      # Borrar puntos que no protegimos (puntos al final de frase)
      doc = re.sub(r'\.', ' ', doc)
      # Borrar guiones SOLO si NO están entre letras o números
      # (Borra guiones decorativos pero mantiene l10n-es o factura-e)
      doc = re.sub(r'(?<![a-zA-Z0-9])-|-(?![a-zA-Z0-9])', ' ', doc)
      # 6. RESTAURAR URLs Y VERSIONES
      for i, ver in enumerate(versiones):
          doc = doc.replace(f' TOKEN_VER_{i} ', f' {ver} ')
      for i, url in enumerate(urls):
          doc = doc.replace(f' TOKEN_URL_{i} ', f' {url} ')
      # 7. COLAPSAR ESPACIOS
      doc = re.sub(r'\s+', ' ', doc).strip()
      return doc


  ### Eliminación signos de puntuación
  def limpieza_selectiva_tecnica(doc):
      """
      Mantiene puntos en todo el texto (para extensiones y URLs).
      Mantiene dos puntos (:) SOLO en URLs.
      Elimina el resto de símbolos técnicos innecesarios.
      """
      if not doc:
          return ""
      # 1. Normalización de espacios y reparación de protocolo
      doc = doc.replace('\\n', ' ').replace('\n', ' ')
      # Reparar URLs mal escritas (ej: https// -> https://)
      doc = re.sub(r'(https?)(//)', r'\1:\2', doc)
      # 2. PROTEGER URLs (Capa de Seguridad Máxima)
      # Esto captura la URL completa con sus : / y .
      urls = re.findall(r'https?://\S+', doc)
      for i, url in enumerate(urls):
          doc = doc.replace(url, f' TOKEN_URL_{i} ')
      # 3. PROTEGER NÚMEROS TÉCNICOS (Versiones y porcentajes)
      # Ej: 16.0, 1.0.1, 10%
      nums_tecnicos = re.findall(r'\d+[.,]\d+(?:\.\d+)*%?', doc)
      for i, nt in enumerate(nums_tecnicos):
          doc = doc.replace(nt, f' TOKEN_NUM_{i} ')
      # 4. LIMPIEZA SELECTIVA DE SÍMBOLOS
      # Eliminamos los dos puntos (:) y la barra (/) aquí porque las URLs ya están protegidas.
      # MANTENEMOS el punto (.) por instrucción directa.
      signos_a_borrar = r'[:/=<>\\[\]{}()|#@*+;\"\'«»`•]'
      doc = re.sub(signos_a_borrar, ' ', doc)
      # 5. TRATAMIENTO DE CONECTORES (Guion y Guion Bajo)
      # Se mantienen solo si están entre letras o números (ej: l10n_es, factura-e)
      # Se borran si son decorativos o están sueltos.
      doc = re.sub(r'(?<![a-zA-Z0-9])[-_]|[-_](?![a-zA-Z0-9])', ' ', doc)
      # 6. RESTAURACIÓN
      # Devolvemos las URLs y Números a su sitio original intactos
      for i, nt in enumerate(nums_tecnicos):
          doc = doc.replace(f' TOKEN_NUM_{i} ', f' {nt} ')
      for i, url in enumerate(urls):
          doc = doc.replace(f' TOKEN_URL_{i} ', f' {url} ')
      # 7. COLAPSAR ESPACIOS
      doc = re.sub(r'\s+', ' ', doc).strip()
      return doc

  #Llamado de función para quitar selectivamente signos de puntuación
  doc = limpieza_selectiva_tecnica(res2)


  ### Tokenización
  doc = nlp2(doc)


  ### Eliminación de stopwords
  doc = [token for token in doc if not token.is_stop]


  ### Lematización
  doc = " ".join([token.lemma_ for token in doc])

  return doc

In [16]:

# ==========================================
# X. MAIN
# ==========================================

path = '/content/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J/data/datos_github.csv'
data = pd.read_csv(path)

coleccion = []
c = 1
for i, doc in enumerate(data['comentarios'], start=1):
    if isinstance(doc, str) and doc.strip().lower().startswith('/ocabot'):
        continue  # Ignorar comentarios del bot
    res = preprocesado(doc)
    c+=1
    if isinstance(res, str) and res.strip():
        coleccion.append(res)
print(f'Se procesaron: {c} documentos')

print(coleccion)


/tmp/ipython-input-2884367730.py:402: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.datetime.utcnow().isoformat()


Se procesaron: 357 documentos
['tratar 4784', ', poder comprobar problema reproducir odoo 17.0 , utilizar módulo l10n_es_aeat_mod190 rama 17.0 . generar fichero modelo 190 odoo 17.0 ejercicio 2025 , sede electrónico aeat devolver error similar , relacionado falta campo obligatorio correspondiente prestación jubilación , viudedad , pensión incapacidad percepción asimilada , requerir aeat ejercicio . actualmente , dicho campo parecer contemplado rama 17.0 módulo , provocar fichero generado válir importación . campo incorporar rama 18.0 , considerar backport cambio 18.0 17.0 16.0 permitir mantener compatibilidad modelo 190 requisito aeat ejercicio 2025 . gracias trabajo soporte . , solución pasar cambio ambos rama . cuestión estar interesado , financiar hacer , esperar hacer , funcionar software abierto contribuir voluntariamente . ideal contribuir parche , hacer versión posible , exigir hacer . parche poner versión intermedio , soler pedir llevar versión superior mantener sincronizado . 

In [17]:

# ==========================================
# 3. VECTORIZACIÓN TF-IDF
# ==========================================


# Parámetros recomendados para repos Git
vectorizer = TfidfVectorizer(
    analyzer="word",
    ngram_range=(1,2),   # unigramas + bigramas; usar (1,1) si solo unigramas
    min_df=2,            # ignora términos que aparecen en menos de 2 documentos
    max_df=0.9,          # ignora términos que aparecen en >90% de docs
    use_idf=True,
    smooth_idf=True,
    norm="l2"
)

# Fit y transformar la colección completa (IDF se calcula aquí)
X_tfidf = vectorizer.fit_transform(coleccion)  # scipy.sparse matrix (n_docs, n_features)
print("Matriz TF-IDF:", X_tfidf.shape)

joblib.dump(vectorizer, "tfidf_vectorizer.joblib")
# Más tarde: vectorizer = joblib.load("tfidf_vectorizer.joblib")

# Vector del documento i (sparse row)
i = 0
vec_i = X_tfidf[i]  # scipy.sparse.csr_matrix (1, n_features)

# Obtener nombres de features (términos / n-gramas)
features = vectorizer.get_feature_names_out()

# Para ver los top-k términos con mayor peso en doc i:
k = 10
row = vec_i.tocoo()
top_idx = np.argsort(row.data)[-k:][::-1]  # índices en row.data ordenados por peso
top_terms = [(features[row.col[idx]], row.data[idx]) for idx in top_idx]
print("Top terms doc", i, top_terms)


Matriz TF-IDF: (285, 1960)
Top terms doc 0 [('tratar', np.float64(1.0))]


In [18]:
print("Matriz TF-IDF:", X_tfidf.shape)
# nuevo_doc debe pasar por el mismo preprocesado que usaste para coleccion
nuevo_doc = "error al ejecutar test de integración en CI"
nuevo_pre = preprocesado(nuevo_doc)  # usar tu función
vec_nuevo = vectorizer.transform([nuevo_pre])  # no fit, solo transform
print(vec_nuevo)

Matriz TF-IDF: (285, 1960)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5 stored elements and shape (1, 1960)>
  Coords	Values
  (0, 297)	0.49655778981256554
  (0, 593)	0.4551664984865397
  (0, 649)	0.2685449115274073
  (0, 1015)	0.5673166696980243
  (0, 1792)	0.3902425486892002


/tmp/ipython-input-2884367730.py:402: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.datetime.utcnow().isoformat()


In [19]:
# 1. Definir rutas
REPO_PATH = f"/content/{DEST_REPO_NAME}"
DATA_FOLDER = os.path.join(REPO_PATH, "data") # O la carpeta que prefieras en el repo
os.makedirs(DATA_FOLDER, exist_ok=True)

# 2. Copiar los archivos desde /content/ al repositorio local clonado
# Asegúrate de que los nombres coincidan exactamente con lo que generó tu código
!cp /content/log_file.txt {DATA_FOLDER}/
!cp /content/tfidf_vectorizer.joblib {DATA_FOLDER}/

# 3. Entrar al directorio del repo y ejecutar comandos Git
%cd {REPO_PATH}

print("📤 Preparando subida de artefactos a GitHub...")

# Agregamos los archivos específicos
!git add data/log_file.txt data/tfidf_vectorizer.joblib

# Hacemos el commit
!git commit -m "Guardado de log y vectorizador joblib"

# Sincronizamos (pull) para evitar conflictos y subimos (push)
!git pull --rebase -X ours origin main
!git push origin main

print("✅ ¡Archivos guardados en el repositorio de GitHub!")

# Regresar al directorio principal de Colab
%cd /content/

/content/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J
📤 Preparando subida de artefactos a GitHub...
[main 88ca0b6] Guardado de log y vectorizador joblib
 2 files changed, 21992 insertions(+)
 create mode 100644 data/log_file.txt
 create mode 100644 data/tfidf_vectorizer.joblib
From https://github.com/jpmachinelearning/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J
 * branch            main       -> FETCH_HEAD
Current branch main is up to date.
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 306.46 KiB | 4.51 MiB/s, done.
Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/jpmachinelearning/U4_S16_Trabajo_Final_2P_PLAZA_A-PADILLA_J.git
   a8139e6..88ca0b6  main -> main
✅ ¡Archivos guardados en el repositorio de GitHub!
/content
